In [ ]:
import re
import json

def extract_nodes_and_edges(filepath):
    with open(filepath, 'r') as f:
        lines = f.readlines()

    nodes = []
    edges = []

    reading_bus = False
    reading_branch = False

    for line in lines:
        line = line.strip()

        # Detect the start of bus data
        if line.startswith('mpc.bus = ['):
            reading_bus = True
            continue
        if reading_bus and line.startswith('];'):
            reading_bus = False
            continue

        # Detect the start of branch data
        if line.startswith('mpc.branch = ['):
            reading_branch = True
            continue
        if reading_branch and line.startswith('];'):
            reading_branch = False
            continue

        # Parse bus lines
        if reading_bus and re.match(r'^\d+', line):
            parts = re.split(r'\s+', line)
            bus_id = int(parts[0])
            voltage = float(parts[9])
            nodes.append({
                "data": {
                    "id": str(bus_id),
                    "voltage": voltage
                }
            })

        # Parse branch lines
        if reading_branch and re.match(r'^\d+', line):
            parts = re.split(r'\s+', line)
            source = str(int(parts[0]))
            target = str(int(parts[1]))
            # Only one direction for undirected graph
            edges.append({
                "data": {
                    "source": source,
                    "target": target
                }
            })

    return {"nodes": nodes, "edges": edges}



filename = input("Please enter the path to the dataset file (e.g., pglib_opf_case30_ieee.m): ")


data = extract_nodes_and_edges(filename)


output_filename = "cytoscape_data.json"; 
with open(output_filename, 'w') as f:
    json.dump(data, f, indent=2)

print(f"Data extracted and saved to {output_filename}")


Data extracted and saved to cytoscape_data.json
